In [34]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

# Inicializa el entorno AMPL con los módulos que necesitas y la licencia UUID
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],  # Selección de módulos
    license_uuid="bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a"  # Tu licencia UUID
)


Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


In [35]:
%%writefile cflp.mod
# Definición de conjuntos
set SR;  # Conjunto de regiones proveedoras
set DR;  # Conjunto de regiones demandantes

# Definición de parámetros
param fixcost{SR};    # Costo fijo para abrir una planta en cada región proveedora
param capacity{SR};   # Capacidad de las plantas en las regiones proveedoras
param demand{DR};     # Demanda en las regiones demandantes
param transp_cost{SR, DR};  # Costo de transporte entre regiones proveedoras y demandantes

# Variables de decisión
var Q{SR, DR} >= 0;  # Cantidad transportada de la región i a la región j
var y{SR} binary;    # Decisión de abrir o no la planta en la región i

# Función objetivo: minimizar el costo total
minimize Totalcost:
    sum{i in SR, j in DR} Q[i,j] * transp_cost[i,j] +
    sum{i in SR} y[i] * fixcost[i];

# Restricciones

# Satisfacer la demanda en todas las regiones demandantes
subject to Demand_Satisfaction{j in DR}:
    sum{i in SR} Q[i,j] == demand[j];

# No exceder la capacidad de las plantas abiertas
subject to Capacity_Limit{i in SR}:
    sum{j in DR} Q[i,j] <= capacity[i] * y[i];

Overwriting cflp.mod


In [36]:
%%writefile Mcflp.mod
# Definición de conjuntos
set SR;  # Conjunto de regiones proveedoras
set DR;  # Conjunto de regiones demandantes

# Definición de parámetros
param fixcost{SR};    # Costo fijo para abrir una planta en cada región proveedora
param capacity{SR};   # Capacidad de las plantas en las regiones proveedoras
param demand{DR};     # Demanda en las regiones demandantes
param transp_cost{SR, DR};  # Costo de transporte entre regiones proveedoras y demandantes

# Variables de decisión
var Q{SR, DR} >= 0;  # Cantidad transportada de la región proveedora i a la región demandante j
var y{SR} binary;    # Decisión de abrir o no la planta en la región i

# Función objetivo: minimizar el costo total
minimize Totalcost:
    sum{i in SR, j in DR} Q[i,j] * transp_cost[i,j] +
    sum{i in SR} y[i] * fixcost[i];

# Restricciones

# Satisfacer la demanda en todas las regiones demandantes
subject to Demand_Satisfaction{j in DR}:
    sum{i in SR} Q[i,j] >= demand[j];  # Se permite que una región demandante sea abastecida por varias plantas

# No exceder la capacidad de las plantas abiertas
subject to Capacity_Limit{i in SR}:
    sum{j in DR} Q[i,j] <= capacity[i] * y[i];

# Restringir las cantidades transportadas solo desde plantas abiertas
subject to Supply_Only_From_Open_Plants{i in SR, j in DR}:
    Q[i,j] <= capacity[i] * y[i];


Overwriting Mcflp.mod


In [37]:
%%writefile cflp.dat
# Conjuntos
set SR := 1 2 3;  # Regiones proveedoras
set DR := A B;    # Regiones demandantes

# Parámetros
param fixcost :=
    1 1000
    2 1500
    3 1200;

param capacity :=
    1 500
    2 700
    3 600;

param demand :=
    A 300
    B 400;

param transp_cost :  A  B :=
    1   4   6
    2   2   3
    3   5   8 ;


Overwriting cflp.dat


In [38]:
%%writefile Mcflp.dat
# Conjuntos
set SR := 1 2 3 4;  # Regiones proveedoras
set DR := A B C;    # Regiones demandantes

# Parámetros
param fixcost :=
    1 1200
    2 1600
    3 1300
    4 1800;

param capacity :=
    1 500
    2 800
    3 600
    4 700;

param demand :=
    A 350
    B 450
    C 300;

param transp_cost :  A  B  C :=
    1   4   6   5
    2   3   2   7
    3   6   5   4
    4   7   3   6 ;


Overwriting Mcflp.dat


In [39]:
%%ampl_eval
model cflp.mod;
data cflp.dat;
option solver gurobi;
solve;
display Q,y;

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 3300
2 simplex iterations
1 branching node
Q :=
1 A     0
1 B     0
2 A   300
2 B   400
3 A     0
3 B     0
;

y [*] :=
1  0
2  1
3  0
;



In [40]:
%%ampl_eval
reset;
model Mcflp.mod;
data Mcflp.dat;
option solver gurobi;
solve;
display Q, y;


Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 6050
8 simplex iterations
1 branching node
Q :=
1 A     0
1 B     0
1 C     0
2 A   350
2 B   450
2 C     0
3 A     0
3 B     0
3 C   300
4 A     0
4 B     0
4 C     0
;

y [*] :=
1  0
2  1
3  1
4  0
;



Inicialización: Se crea una población inicial de individuos aleatorios.
Evaluación: Se evalúa cada individuo utilizando la función de aptitud.
Selección: Se seleccionan los individuos más aptos para reproducirse.
Cruce: Los individuos seleccionados se combinan para formar nuevos individuos.
Mutación: Se mutan algunos individuos aleatoriamente para garantizar la diversidad.
Reemplazo: Los individuos de la nueva generación reemplazan a la población anterior.

Repetición: Los pasos del 2 al 6 se repiten hasta que se cumpla la condición de parada.

In [42]:
import numpy as np

def algoritmo_genetico(num_instalaciones, num_clientes, capacidades, costos_apertura, costos_asignacion, generaciones=100, tasa_mutacion=0.1):
    
    poblacion = np.random.randint(2, size=(10, num_instalaciones, num_clientes)) 

    def evaluacion(solucion): 
        
        costo_total = 0
        instalaciones_abiertas = np.zeros(num_instalaciones)

        for cliente in range(num_clientes):
            for instalacion in range(num_instalaciones):
                if solucion[instalacion][cliente] == 1:
                    costo_total += costos_asignacion[instalacion][cliente]
                    instalaciones_abiertas[instalacion] += 1

       
        for i in range(num_instalaciones):
            if instalaciones_abiertas[i] > 0:
                costo_total += costos_apertura[i]

       
        for i in range(num_instalaciones):
            if instalaciones_abiertas[i] > capacidades[i]:
                return float('inf')  

        return costo_total

    def seleccion(poblacion):
       
        aptitudes = [evaluacion(solucion) for solucion in poblacion]
        indices = np.argsort(aptitudes)[:2] 
        return poblacion[indices]

    def cruce(padre1, padre2):
        
        punto_cruce = np.random.randint(1, num_clientes)
        hijo = np.zeros_like(padre1)
        for i in range(num_instalaciones):
            hijo[i][:punto_cruce] = padre1[i][:punto_cruce]
            hijo[i][punto_cruce:] = padre2[i][punto_cruce:]
        return hijo

    def mutacion(solucion):
      
        for i in range(num_instalaciones):
            if np.random.random() < tasa_mutacion:
                
                cliente = np.random.randint(num_clientes)
                solucion[i][cliente] = 1 - solucion[i][cliente] 
        return solucion

    def reemplazo(poblacion, hijo):
       
        aptitudes = [evaluacion(solucion) for solucion in poblacion]
        menos_aptos_index = np.argmin(aptitudes)
        poblacion[menos_aptos_index] = hijo

    def repeticion(poblacion):
        for generacion in range(generaciones):
            print(f"Generación {generacion + 1}:")
            padres = seleccion(poblacion)
            hijo = cruce(padres[0], padres[1])
            hijo = mutacion(hijo)
            reemplazo(poblacion, hijo)
            mejor_costo = evaluacion(padres[0])
            print(f"Mejor costo: {mejor_costo}")

    print("Población inicial:")
    print(poblacion)

    repeticion(poblacion)


# Datos actualizados
num_instalaciones = 3  # Número de instalaciones
num_clientes = 2       # Número de clientes
capacidades = [500, 700, 600]  # Capacidades de las instalaciones
costos_apertura = [1000, 1500, 1200]  # Costos de apertura
costos_asignacion = np.random.randint(1, 50, size=(num_instalaciones, num_clientes))  # Costos de asignación aleatorios

algoritmo_genetico(num_instalaciones, num_clientes, capacidades, costos_apertura, costos_asignacion)


Población inicial:
[[[1 1]
  [1 1]
  [0 0]]

 [[0 0]
  [1 0]
  [0 1]]

 [[1 0]
  [0 0]
  [0 0]]

 [[1 1]
  [1 1]
  [1 0]]

 [[1 1]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [1 0]]

 [[1 0]
  [1 0]
  [1 0]]

 [[0 0]
  [1 1]
  [0 0]]

 [[0 1]
  [1 0]
  [1 1]]

 [[1 1]
  [1 0]
  [1 1]]]
Generación 1:
Mejor costo: 1007
Generación 2:
Mejor costo: 1054
Generación 3:
Mejor costo: 1007
Generación 4:
Mejor costo: 1202
Generación 5:
Mejor costo: 1558
Generación 6:
Mejor costo: 2257
Generación 7:
Mejor costo: 1007
Generación 8:
Mejor costo: 1007
Generación 9:
Mejor costo: 1007
Generación 10:
Mejor costo: 1007
Generación 11:
Mejor costo: 1007
Generación 12:
Mejor costo: 1007
Generación 13:
Mejor costo: 1054
Generación 14:
Mejor costo: 2521
Generación 15:
Mejor costo: 2551
Generación 16:
Mejor costo: 2598
Generación 17:
Mejor costo: 2598
Generación 18:
Mejor costo: 2598
Generación 19:
Mejor costo: 2598
Generación 20:
Mejor costo: 2612
Generación 21:
Mejor costo: 2717
Generación 22:
Mejor costo: 1558
Gene